#  (version-02) Content based filltering movie recommendation system
### lemmatization
### SentenceTransformer(one per movie)
### Embedding(384) + Cosine similarity
### No individual weighting of features

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ast # to convert string into list

from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches
import joblib

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger')  # needed for pos_tag
nltk.download('averaged_perceptron_tagger_eng')

from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lapmart\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lapmart\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lapmart\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lapmart\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Lapmart\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [144]:
# !pip install sentence-transformers

In [145]:
movies = pd.read_csv(r'D:\Ai Projects\Movie Rec System - Content based\datasets\tmdb_5000_movies.csv') # 'r' for raw string, o/w use / or \\
credits = pd.read_csv(r'D:\Ai Projects\Movie Rec System - Content based\datasets\tmdb_5000_credits.csv')

In [146]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [147]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [148]:
movies.shape

(4803, 20)

In [149]:
credits.shape

(4803, 4)

## Merge movie, credits into one dataframe

In [150]:
movies = movies.merge(credits, on='title')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [151]:
movies.shape

(4809, 23)

In [152]:
# movies.value_counts('original_language')

## Let choose columns for metrics

In [153]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [154]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']] # [[ ]]  for multiple cvolumns together
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [155]:
movies.shape

(4809, 7)

## Overview & data cleaning

In [156]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [157]:
movies.dropna(inplace=True)

In [158]:
movies.isnull().sum()
movies.shape

(4806, 7)

In [159]:
movies.duplicated().sum()

np.int64(0)

In [160]:
print(movies.iloc[0])

movie_id                                                19995
title                                                  Avatar
overview    In the 22nd century, a paraplegic Marine is di...
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object


## Data Cleaning Functions

In [161]:
# we have some column records as list but here they are consdidered as string so let convert them into original list

# convert string list into list for fetchign specific df, always pass a string
def string_to_list_convertor(text, no_of_items=None):

    # text is null return []
    if isinstance(text, float) and pd.isna(text):
        return []
        
    name_list = []
    count=1
    
    for i in ast.literal_eval(text):
        name_list.append(i['name'])
        if no_of_items is not None:
            if count >= no_of_items:
                break
            count += 1
            
    return name_list

# fetch director name, always pass a string
def fetch_director(text):
    
    # text is null return []
    if pd.isna(text):
        return []
  
    director_list = []

    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            director_list.append(i['name'])
            break
    return director_list

# Reomove spaces betweeen names, always pass a list
def remove_spaces(text):
    names = []

    for i in text:
        names.append(i.replace(" ", ""))
    return names


In [162]:
# movies['keywords'][0]

In [163]:
# movies.iloc[0]['crew']

In [164]:
movies['genres'] = movies['genres'].apply(string_to_list_convertor).apply(remove_spaces)
movies['cast'] = movies['cast'].apply(lambda x :  string_to_list_convertor(x, 3)).apply(remove_spaces)
movies['keywords'] = movies['keywords'].apply(string_to_list_convertor).apply(remove_spaces)
movies['crew'] = movies['crew'].apply(fetch_director).apply(remove_spaces)

movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [165]:
movies.iloc[0]['overview']

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [166]:
# seperate each word, put into list
movies['overview'] = movies['overview'].apply(lambda x : x.split())
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [167]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [168]:
new_df = movies[['movie_id', 'title', 'tags']].copy()
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [169]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x).lower())
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [170]:
new_df.iloc[0]['tags']

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

## Lemmatization
    This is bettter than stemming
    has - have
    allowed - allow (with the help of pos_tag)

In [171]:
# get text(String) ---> text(list) ---> apply lemmatization for exh word ---> join bakc to string
# lemmatization & lower the texts
# lemmatization uses linguistic rules ---> ["running" → "running", "wolves" → "wolf", "better" → "good"]

lemmatizer = WordNetLemmatizer()

# helper function to map POS tags
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,   # adjective
        "N": wordnet.NOUN,  # noun
        "V": wordnet.VERB,  # verb
        "R": wordnet.ADV    # adverb
    }
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):

    # input must be a str
    if not isinstance(text, str):
        return "" 

    # tokanize : textd into lower case & split into words list
    # Splits text into words and separates punctuation as tokens
    # Hello! How are you? ---> ['Hello', '!', 'How', 'are', 'you', '?']
    words = word_tokenize(text) # from NLTK

    lemmatized_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]
        
    return " " .join(lemmatized_words) #str

# " ".join([lemmatizer.lemmatize(word) for word in words])


In [172]:
new_df['tags'] = new_df['tags'].apply(lemmatize_text)
new_df.iloc[0]['tags']

'in the 22nd century , a paraplegic marine be dispatch to the moon pandora on a unique mission , but becomes torn between follow order and protect an alien civilization . action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

# Sentence Embeddings (Semantic Content-Based)

### Embedding
    An embedding is just a way to turn something like text, images, or items into a vector of numbers (a list of floats) so that a computer can understand and work with it mathematically.
    
    For example:
    Text: "Superman saves the world" → [0.12, -0.03, 0.45, ..., 0.01]
    Text: "Lex Luthor plots revenge" → [0.10, -0.01, 0.50, ..., -0.05]

    These vectors (embeddings) capture the meaning of the text. Sentences with similar meaning will have vectors that are closer together in the vector space.

    embeddings = model.encode(new_df['tags'].tolist())
    Each movie’s tags are converted into a vector of 384 numbers (for this model).

### Compute embeddings

In [173]:
 # Loads a pre-trained language model, This model already “knows” how to turn sentences or words into embeddings

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(new_df['tags'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/151 [00:00<?, ?it/s]

In [174]:
embeddings.shape

(4806, 384)

In [175]:
emb_df = pd.DataFrame(embeddings) # pandas-style view
emb_df.head() # Each embedding has 384 length 1D vector for each data tag column

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.021674,-0.003296,0.006376,-0.061012,-0.092974,-0.008063,-0.006084,-0.091690,0.013168,0.013109,...,0.028674,0.078727,-0.031640,0.133622,-0.021232,0.036238,0.058831,0.019312,0.005408,-0.000012
1,-0.057014,-0.082991,-0.066531,-0.022505,-0.004650,0.052054,0.023929,-0.061275,0.026862,0.002593,...,0.009442,0.081700,-0.062290,0.138458,0.011710,0.025898,0.048936,-0.124824,-0.017645,-0.034519
2,-0.150658,-0.016722,-0.038641,-0.031952,-0.042751,-0.010248,0.037981,0.016800,0.028525,0.004727,...,0.041861,0.049694,0.001519,0.014255,-0.050001,-0.013155,0.001062,-0.056420,-0.011900,-0.015049
3,-0.031494,-0.003712,-0.086022,-0.041501,0.004759,0.001501,-0.044487,0.046638,0.011271,0.042558,...,0.026412,-0.013720,0.026372,-0.021524,-0.054985,0.011257,0.084077,-0.055615,0.071067,0.055433
4,-0.037790,-0.018044,0.011908,-0.026298,-0.027477,-0.015985,0.092248,0.006484,0.011311,0.056437,...,0.128305,0.032339,0.004140,0.102640,0.013419,-0.010733,-0.040642,-0.051671,-0.066316,-0.015572


### Compute cosine similarity

In [176]:
# Gives similarity between every pair of movies
# Can pick top-k similar movies for recommendations.

similarity_matrix = cosine_similarity(embeddings) # similarity_matrix[i][j] = similarity between movie i and movie j

### Recommendation function

In [177]:
def recommend(movie, top_k =5):
    # all titles
    titles = new_df['title'].tolist()
    
    # Find the closest match (allowing typos / case differences)
    matches = get_close_matches(movie, titles, n=1, cutoff=0.6)

    # Checks if the matches list is empty
    if not matches:
        print(f"No close match found for '{movie}'")
        return

    best_match = matches[0]
    # Finds the row number of the movie in new_df whose title exactly matches best_match
    index = new_df[new_df['title'] == best_match].index[0]

    distance = sorted(list(enumerate(similarity_matrix[index])), reverse=True, key= lambda x : x[1])

    # ignore first value (1 is itself same movie)
    print(f"\nResults for: {best_match}\n")
    for i in distance[1:top_k+1]:
        print(new_df.iloc[i[0]].title) # i[0] → movie_index in the DataFrame, access row by integer position, fetch the title column from that row


In [178]:
recommend('batman', 10)


Results for: Batman

Batman
The Dark Knight
Batman Forever
Batman Begins
The Dark Knight Rises
Batman & Robin
Batman v Superman: Dawn of Justice
Batman Returns
Batman: The Dark Knight Returns, Part 2
Brick Mansions


## Saving importants as pkl

In [180]:
os.makedirs('artifacts_v2', exist_ok=True)

joblib.dump(new_df, 'artifacts_v2/movie_list.pkl')
joblib.dump(similarity_matrix, 'artifacts_v2/similarity_matrix.pkl')

['artifacts_v2/similarity_matrix.pkl']